In [1]:
import helpers
from helpers import *
print(least_busy)

ibmqx2


In [2]:
gate_times_melbourne = [
        ('u1', None, 0), ('u2', None, 100), ('u3', None, 200),
        ('cx', [1, 0], 678), ('cx', [1, 2], 547), ('cx', [2, 3], 721),
        ('cx', [4, 3], 733), ('cx', [4, 10], 721), ('cx', [5, 4], 800),
        ('cx', [5, 6], 800), ('cx', [5, 9], 895), ('cx', [6, 8], 895),
        ('cx', [7, 8], 640), ('cx', [9, 8], 895), ('cx', [9, 10], 800),
        ('cx', [11, 10], 721), ('cx', [11, 3], 634), ('cx', [12, 2], 773),
        ('cx', [13, 1], 2286), ('cx', [13, 12], 1504), ('cx', [], 800)
    ]

noise_model_melbourne = noise.device.basic_device_noise_model(melbourne.properties(), gate_times=gate_times_melbourne)
basis_gates_melbourne = noise_model_melbourne.basis_gates
coupling_map_melbourne = melbourne.configuration().coupling_map

### W8 O(n)

In [3]:
qreg_lin = QuantumRegister(8)
w8_lin = QuantumCircuit(qreg_lin)
w8_lin.x(qreg_lin[0])
Bdirect(w8_lin, qreg_lin, 1, 0, 1/8)
Bdirect(w8_lin, qreg_lin, 2, 1, 1/7)
Bdirect(w8_lin, qreg_lin, 3, 2, 1/6)
Bdirect(w8_lin, qreg_lin, 4, 3, 1/5)
Bdirect(w8_lin, qreg_lin, 5, 4, 1/4)
Bdirect(w8_lin, qreg_lin, 6, 5, 1/3)
Bdirect(w8_lin, qreg_lin, 7, 6, 0.5)
print(w8_lin)
job_lin = qiskit.execute(w8_lin, Aer.get_backend('statevector_simulator'))
theorical_psi_lin = job_lin.result().get_statevector(w8_lin)
print(theorical_psi_lin)

                ┌───┐                                ┌───┐     »
q0_0: |0>───────┤ X ├──────────■─────────────────────┤ X ├─────»
         ┌──────┴───┴───────┐┌─┴─┐┌─────────────────┐└─┬─┘     »
q0_1: |0>┤ U3(-0.36137,0,0) ├┤ X ├┤ U3(0.36137,0,0) ├──■────■──»
         ├─────────────────┬┘└───┘└─────────────────┘     ┌─┴─┐»
q0_2: |0>┤ U3(-0.3876,0,0) ├──────────────────────────────┤ X ├»
         ├─────────────────┴┐                             └───┘»
q0_3: |0>┤ U3(-0.42053,0,0) ├──────────────────────────────────»
         ├──────────────────┤                                  »
q0_4: |0>┤ U3(-0.46365,0,0) ├──────────────────────────────────»
         ├─────────────────┬┘                                  »
q0_5: |0>┤ U3(-0.5236,0,0) ├───────────────────────────────────»
         ├─────────────────┴┐                                  »
q0_6: |0>┤ U3(-0.61548,0,0) ├──────────────────────────────────»
         ├─────────────────┬┘                                  »
q0_7: |0>┤ U3(-0.7854,0,0

In [4]:
tomo_circuits = state_tomography_circuits(w8, qreg)
job = qiskit.execute(tomo_circuits,
                     simulator,
                     noise_model=noise_model_melbourne,
                     coupling_map=coupling_map_melbourne,
                     basis_gates=basis_gates_melbourne)
job_monitor(job, monitor_async = True)

calib_circuit, state_labels = mc.complete_meas_cal(qr=qreg)
job_cal = qiskit.execute(calib_circuit,
                         simulator,
                         noise_model=noise_model_melbourne,
                         coupling_map=coupling_map_melbourne,
                         basis_gates=basis_gates_melbourne)
job_monitor(job_cal, monitor_async=True)

In [ ]:
job_results = job.result()
cal_results = job_cal.result()
meas_fitter = mc.CompleteMeasFitter(cal_results, state_labels)
correct_results = meas_fitter.filter.apply(job_results)
fitter = StateTomographyFitter(correct_results, tomo_circuits)
fitted_rho = fitter.fit()
fidelity = state_fidelity(theorical_psi, fitted_rho)
print(fidelity)